### Définition du problème :

1. **Univers d’investissement et indices**

    - Un univers d’investissement composé de 100 valeurs

    - Des indices constitués de ces 100 valeurs

        - I1 : 100 valeurs équipondérées (1% chacune)
        - I2 : 5 valeurs valent chacune 10% et les 95 autres sont équipondérées pour représenter 50%
        - I3 : 1 valeur vaut 50% et les 99 autres sont équipondérées pour représenter 50%

2. **Une méthode d’investissement de génération d’alpha**

    - Une méthode de détection de momentum (hausse / baisse) avec une probabilité d’occurrence de 60%

3. **Constitution d’un portefeuille de 30 valeurs**

**Question 1** : Quelle est, avec cette méthode d’investissement de génération d’alpha,  la ou les constitutions de portefeuille optimales pour battre sur le long terme l’indice I1 ? l’indice I2 ? l’indice I3 ?

**Question 2** : Trouver une méthode générique en fonction des allocations de l’indice de référence à battre





### Proposition de méthodologie :


Pour tenter de proposer une méthode générique, nous allons aborder le sujet de manière théorique.

Nous allons aborder le sujet comme si nous avions $N_{indice}$ valeurs dans l'indice, et $N_{portefeuille}$ dans le portefeuille. Notre univers à une probabilité $p_h$ de rendements haussiers $r_h$, et $1 - p_h$ de rendements baissiers $r_b$. Nous avons également un outil de prédiction de momentum qui a une accuracy de $p_{pc}$%. 

Dans un souci de clarté, les valeurs équipondérées seront appelées des "SmallCap", et les valeurs avec une plus forte pondération seront nommées les "BigCap".

Dans notre indice on peut donc avoir $n_{BigCap}$ qui font toutes un poids de $w_{BigCap}$. 

De la sorte, on a univers commun à tous les indices : 

- $N_{indice} = 100$ 
- $N_{portefeuille} = 30$
- $p_h = 0.5$ 
- $p_{pc} = 0.6$ 
- $r_h = 1$
- $r_b = -1$

Et on a les paramètres suivant pour : 

**L'indice 1** : 

- $n_{BigCap} = 0$
- $w_{BigCap} = 0$


**L'indice 2** : 

- $n_{BigCap} = 5$
- $w_{BigCap} = 0.1$

**L'indice 3** : 

- $n_{BigCap} = 1$
- $w_{BigCap} = 0.5$

Pour être capable de battre l'indice, on va faire l'hypothèse que la meilleure manière de battre un équipondéré sans informations supplémentaire à ce que nous avons, c'est de faire un équipondéré. 
On va donc faire le choix de garder $N_{portefeuille} - n_{BigCap}$ en "SmallCap", et de prendre nos $n_{BigCap}$ dans le portefeuille. Une proposition pour tirer profit de notre capacité de prédiction, serait de sur-pondérer de $x$ les "Big Cap"  qu'on pense haussières, et de sous-pondérer de $y$ les "Big Cap" qu'on pense baissières. De la sorte, nous pourrons traiter tous les cas que nous voulons (On peut imaginer sous-pondérer de $y$ avec $y = w_{BigCap}$ si on souhaite ne pas prendre nos "BigCap", etc...)


Donc la question de l'optimisation de notre performance va se faire à travers l'optimisation de $x$ et $y$, savoir si nous prenons ou pas une BigCap, et à quelle pondération, en fonction de notre pouvoir de prédiction.

### Définition des distributions de l'indice :

Vous pourrez trouver le détail de ces calculs, ainsi que les rappels des fondamentaux sur les calculs d'espérance et de variance, dans un autre Jupyter Notebook plus détaillé. Chacune des équations a été testée en parallèle d'une simulation pour vérifier si les résultats théoriques sont bien en adéquation avec la pratique.

Notre rendement d'indice est : $R_{indice} = Sa + B$, avec : 

**$S$ : La distribution de rendement de nos "SmallCap", soit $N_{indice} - n_{BigCap}$ valeurs.**

- L'espérance de rendement de cette distribution est simple à calculer :  

    - Nous avons une probabilité $p_h$ d'avoir un rendement $r_h$.
    - Une probabilité $1 - p_h$ d'avoir un rendement $r_b$. 

Donc : 

$$E(S) = p_h \times r_h + (1 - p_h) \times r_b$$

- Et sa variance : 

$$Var(S) = \frac {p_h \times r_h^2 + (1 - p_h) \times r_b^2 - E(S)^2}{N_{indice} - n_{BigCap}}$$

**$B$ : La distribution de rendement de nos "BigCap", soit $n_{BigCap}$ valeurs.**

- L'espérance de rendement de cette distribution est tout aussi simple. Nous avons :

    - une probabilité $p_h$ d'avoir un rendement $r_h$ avec un poids de $w_{BigCap}$.
    - une probabilité $1 - p_h$ d'avoir un rendement $r_b$ avec un poids de $w_{BigCap}$ 

et sachant qu'on en a $n_{BigCap}$. Donc : 

$$E(B) = (p_h \times r_h + (1 - p_h) \times r_b) \times w_{BigCap} \times n_{BigCap}$$

- Et sa variance : 

$$Var(B) = [(p_h \times r_h^2 + (1 - p_h) \times r_b^2) \times w_{BigCap}^2 - ((p_h \times r_h + (1 - p_h) \times r_b) \times w_{BigCap})^2] \times n_{BigCap}$$

**$a$ : Une constante qui a pour vocation d'ajuster le poids du rendement de la distribution des SmallCap en fonction de $n_{BigCap}$ et $w_{BigCap}$. Soit**

$$a = 1 - w_{BigCap} \times n_{BigCap}$$

### Déterminer l'espérance et la variance de $R_{indice}$ :

Donc :

$$E(R_{indice}) = E(S)a + E(B)$$

et sachant qu'il n'y pas de dépendance entre la distribution S et B : 

$$Var(R_{indice}) = Var(S)a^2 + Var(B)$$

### Définition des distributions du portefeuille :

Notre rendement de portefeuille est : $R_{portefeuille} = XW + Y$, avec : 

**$X$ : La distribution de rendement de nos "SmallCap" séléctionnés selon notre pouvoir de prédiction, soit $N_{portefeuille} - n_{BigCap}$ valeurs.**

La méthodologie est la suivante, nous récupérons $N_{portefeuille} - n_{BigCap}$ valeurs que nous avons prédis haussières. 

- Nous avons : 

    - Notre prédiction est correcte $\frac{p_h \times p_{pc}}{p_h \times p_{pc} + (1 - p_h) \times (1 - p_{pc})}$  et nous avons une hausse $r_b$
    - Notre prédiction est incorrecte $\frac{(1 - p_h) \times (1 - p_{pc})}{p_h \times p_{pc} + (1 - p_h) \times (1 - p_{pc})}$  et nous avons une hausse $r_b$

Donc : 

$$E(X) = \left( \frac{p_h \times p_{pc}}{p_h \times p_{pc} + (1 - p_h) \times (1 - p_{pc})} \right) \cdot r_h + \left( \frac{(1 - p_h) \times (1 - p_{pc})}{p_h \times p_{pc} + (1 - p_h) \times (1 - p_{pc})} \right) \cdot r_b$$


- Et sa variance : 

$$Var(X) = \frac {\left( \frac{p_h \times p_{pc}}{p_h \times p_{pc} + (1 - p_h) \times (1 - p_{pc})} \right) \cdot r_h^2 + \left( \frac{(1 - p_h) \times (1 - p_{pc})}{p_h \times p_{pc} + (1 - p_h) \times (1 - p_{pc})} \right) \cdot r_b^2 - E(X)^2}{N_{portefeuille} - n_{BigCap}}$$

**$Y$ : La distribution de rendement de nos "BigCap" ajustés par la sur-pondération ou sous-pondération, soit $n_{BigCap}$ valeurs avec un poids de $w_{BigCap} + x$ ou $w_{BigCap} - y$**

- L'espérance de rendement de cette distribution est la suivante :

    - une probabilité $p_h$ d'avoir un rendement $r_h$ et de bien le prédire, donc $p_h \times p_{pc} \times (w_{BigCap} + x) \times r_h$
    - une probabilité $p_h$ d'avoir un rendement $r_h$ et de mal le prédire, donc $p_h \times (1 - p_{pc}) \times (w_{BigCap} - y) \times r_h$
    - une probabilité $(1 - p_h)$ d'avoir un rendement $r_b$ et de bien le prédire, donc $(1 - p_h) \times p_{pc} \times (w_{BigCap} - y) \times r_b$
    - une probabilité $(1 - p_h)$ d'avoir un rendement $r_h$ et de mal le prédire, donc $(1 - p_h) \times (1 - p_{pc}) \times (w_{BigCap} + x) \times r_b$

et sachant qu'on en a $n_{BigCap}$. Donc : 

$$E(Y) = [p_h \times r_h \times (p_{pc} \times (w_{BigCap} + x) + (1 - p_{pc}) \times (w_{BigCap} - y)) + (1 - p_h)\times r_b \times (p_{pc} \times (w_{BigCap} - y) + (1 - p_{pc}) \times (w_{BigCap} + x))]\times n_{BigCap}$$

- Et sa variance : 

$$Var(Y) = [p_h \times r_h^2 \times (p_{pc} \times (w_{BigCap} + x)^2 + (1 - p_{pc}) \times (w_{BigCap} - y)^2) + (1 - p_h)\times r_b^2 \times (p_{pc} \times (w_{BigCap} - y)^2 + (1 - p_{pc}) \times (w_{BigCap} + x)^2) $$
$$ - (p_h \times r_h \times (p_{pc} \times (w_{BigCap} + x) + (1 - p_{pc}) \times (w_{BigCap} - y)) + (1 - p_h)\times r_b \times (p_{pc} \times (w_{BigCap} - y) + (1 - p_{pc}) \times (w_{BigCap} + x)))] \times n_{BigCap}$$

**$W$ : La distribution de l'ajustement de la poche X, ajustér par les sur-pondérations ou sous-pondérations faites sur Y**

Comment on l'a vu précédemment, on a : 

- une probabilité $p_h$ d'avoir un rendement $r_h$ et de bien le prédire, donc $p_h \times p_{pc} \times (w_{BigCap} + x) \times r_h$, donc un surajustement de $p_h \times p_{pc} \times x$
- une probabilité $p_h$ d'avoir un rendement $r_h$ et de mal le prédire, donc $p_h \times (1 - p_{pc}) \times (w_{BigCap} - y) \times r_h$ donc un surajustement de $p_h \times (1 - p_{pc}) \times y$
- une probabilité $(1 - p_h)$ d'avoir un rendement $r_b$ et de bien le prédire, donc $(1 - p_h) \times p_{pc} \times (w_{BigCap} - y) \times r_b$ donc un surajustement de $(1 - p_h) \times p_{pc} \times y$
- une probabilité $(1 - p_h)$ d'avoir un rendement $r_h$ et de mal le prédire, donc $(1 - p_h) \times (1 - p_{pc}) \times (w_{BigCap} + x) \times r_b$ donc un surajustement de $(1 - p_h) \times (1 - p_{pc}) \times x$

Donc :

$$E(W) = (1 - n_{BigCap} \times (w_{BigCap} + (p_h * p_{pc}+ (1 - p_h) * (1 - p_{pc})) * x - (p_h * (1 - p_{pc}) + (1 - p_h) * ppc) * y)$$

Et sa variance : 

$$Var(W) = (1 - n_{BigCap} \times (w_{BigCap} + (p_h * p_{pc} + (1 - p_h) * (1 - p_{pc})) * x^2 - (p_h * (1 - p_{pc}) + (1 - p_h) * ppc) * y^2)) - E(W)^2$$



### Déterminer l'espérance et la variance de $R_{portefeuille}$ :

Finalement, avec nos espérances et variances terminés, on peut commencer à calculer l'espérance de $R_{portefeuille} = XW + Y$.

$$E(R_{portefeuille}) = E(X \times W + Y) = E(X \times W) + E(Y) = E(X) \times E(W) + E(Y)$$

La difficulté réside plus dans le calcul de la variance : 

$$Var(R_{portefeuille}) = Var(X \times W + Y)$$

Hors il y a une dépendance entre la distribution $W$ et $Y$, puisque l'ajustement dans la distribution $W$ dépend directment des valeurs dans $Y$, donc : 

$$Var(X \times W + Y) = Var(X \times W) + Var(Y) + 2 Cov(X \cdot W, Y)$$

Hors X et W sont indépendants, alors :

$$Var(X \times W) = E(X)^2 \times Var(W) + E(W)^2 \times Var(X) + Var(X) + Var(W)$$

donc :

$$Var(X \times W + Y) = E(X)^2 \times Var(W) + E(W)^2 \times Var(X) + Var(X) + Var(W)  + Var(Y) + 2 Cov(X \cdot W, Y)$$

Il nous reste donc à définir la covariance : 
$$Cov(X \cdot W, Y) = E((X \cdot W) \cdot Y) - E(X \cdot W)E(Y)$$ 

Hors X et W sont indépendants, donc $E(X \cdot W) = E(X)E(W)$ : 

$$Cov(X \cdot W, Y) = E(X \cdot W \cdot Y) - E(X)E(W)E(Y)$$

Comme expliqué dans le contexte, on suppose X indépendant de W et de Y, et W et Y dépendants. Alors : 

$$E(X \cdot W \cdot Y) = E(X \cdot (W \cdot Y)) = E(X) E(W \cdot Y)$$

On a peut donc définir la covariance tel que : 

$$Cov(X \cdot W, Y) = E(X)E(W \cdot Y) - E(X)E(W)E(Y)$$

Voici la formule finale de la variance : 

$$Var(X \times W + Y) = E(X)^2 \times Var(W) + E(W)^2 \times Var(X) + Var(X) + Var(W)  + Var(Y) + 2[E(X)E(W \cdot Y) - E(X)E(W)E(Y)]$$

La seule inconnu dans cette équation est $E(W \cdot Y)$, il nous reste donc à la définir.

On fait une proposition de solution pour déterminer $E(W \cdot Y)$ dans le fichier "Sujet de Fond - Covariance".

### Déterminer l'espérance et la variance du delta entre $R_{portefeuille}$ et $R_{indice}$ :

On cherche à définir l'espérance et la variance du delta entre le rendement du portefeuille et celui de l'indice.

Pour l'espérance, la marche à suivre est assez simple, d'après la linéarité de l'espérance :

$$E(\Delta R) = E(R_{portefeuille} - R_{indice})$$

$$E(\Delta R) = E(R_{portefeuille}) - E(R_{indice})$$ 

Hors $E(R_{portefeuille})$ et $E(R_{indice})$ sont déjà des variables que nous avons déterminées précédemment.

Pour la variance, la tâche est un peu plus complexe. Nous pouvons déterminer la variance de la sorte :

$$Var(\Delta R) = Var(R_{portefeuille} - R_{indice})$$

$$Var(\Delta R) = Var(R_{portefeuille}) + Var(R_{indice}) - 2 \cdot Cov(R_{portefeuille}, R_{indice})$$

Nous allons nous heurter au même problème que pour déterminer $Var(R_{portefeuille})$ précédemment. Nous allons donc détailler notre calcul.

$$Var(\Delta R) = Var(XW + Y) + Var(S(1 - n \cdot w) + B) - 2 \cdot Cov(XW + Y, S(1 - n \cdot w) + B)$$

Notre inconnu ici est la covariance entre le portefeuille et l'indice :

$$Cov(XW + Y, aS + B) = E[(XW + Y) \cdot (aS + B)] - E(XW + Y) \times E[(aS + B)]$$

La variable à définir est la suivante : $E[(XW + Y) \cdot (aS + B)]$. Nous allons la développer :

$$E[(XW + Y) \cdot (aS + B)] = E(XWaS + XWB + YaS + YB)$$

$$= E(XWaS) + E(XWB) + E(YaS) + E(YB)$$

$$= a \cdot E(XWS) + E(XWB) + a \cdot E(YS) + E(YB)$$

Pour continuer, nous allons nous confronter à la question de la dépendance de nos distributions. 

Après des tests de corrélation sur des smilations, on va faire l'hypothèse suivante sur nos distributions : 

- X et aS sont dépendantes. 
- Y, B et W sont dépendantes.

La majeur partie de la covariance vient de la dépendance entre Y et B. 

Maintenant que nous avons nos hypothèses de relation de dépendance, nous pouvons continuer de développer $E[(XW + Y) \cdot (aS + B)]$ :

$$E[(XW + Y) \cdot (aS + B)] = a \cdot E(XWS) + E(XWB) + a \cdot E(YS) + E(YB)$$

$$= a \cdot E(W) \cdot E(XS) + E(X) \cdot E(WB) + a \cdot E(Y) \cdot E(S) + E(YB)$$

Les trois variables à définir ici sont $E(XS)$, $E(WB)$ et $E(YB)$. 

Pour le moment $E(XS)$ semble trop compliqué à définir. On décide de le négliger pour l'instant, puisque X et S ne sont pas liées à x et y, ni a nos BigCap, donc même si nos résultats est légèrement différent de la simulation, l'impacte sur l'optimisation de nos pondération devrait être moindre. On pourra toujours simulée unique $E(XS)$ pour trouver un résultat très proche de la réalité (Bien que le temps de calcul de ce procédé peut rapidement être élevé et rendre chronophage la tâche, en plus d'être imprécise).

Pour $E(WB)$ et $E(YB)$ nous pouvons réutiliser le procéder utilisé pour $E(WY)$ que nous avons détaillé dans la partie sur la covariance pour la variance du rendement du portefeuille. 




### Résultats et visualisation : 

Maintenant que nous avons :

- $E(R_{portefeuille})$
- $Var(R_{portefeuille})$

- $E(R_{indice})$
- $Var(R_{indice})$

- $E(R_{portefeuille} - R_{indice})$
- $Var(R_{portefeuille} - R_{indice})$

Nous pouvons analyser les résultats. 

Nous importer les fonctions de calculs et de simulations.

In [1]:
from itertools import combinations_with_replacement
from collections import Counter
import pandas as pd
import numpy as np
from math import factorial

##############################################
# Portefeuille ou indice
##############################################

def portfolio(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1, indice = False):
    
    if indice == False:
        # Espérance des Small Cap
        E_R_sc_adjusted = smallcap_ptf(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[0]

        # Espérance des Big Cap 
        E_R_bc_total = bigcap_ptf(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[0]

        # Espérance du portefeuille
        E_R = E_R_sc_adjusted + E_R_bc_total

        # Variance du small cap
        Var_R_sc_adjusted = smallcap_ptf(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[1]

        # Variance du big cap
        Var_R_bc_total = bigcap_ptf(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[1]
        
        # Espérance des Small Cap sans ajustement
        E_R_sc_ptf_total = smallcap_ptf(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[2]

        E_WY = esperance_WY(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)

        # Covariance
        Cov_R_sc_bc = E_R_sc_ptf_total * E_WY - E_R_sc_adjusted * E_R_bc_total

        # Variance du portefeuille
        Var_R = Var_R_sc_adjusted + Var_R_bc_total + 2 * Cov_R_sc_bc
    
    else:
        # Espérance des Small Cap
        E_R_sc_adjusted = smallcap_indice(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[0]

        # Espérance des Big Cap 
        E_R_bc_total = bigcap_indice(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[0]

        # Espérance du portefeuille
        E_R = E_R_sc_adjusted + E_R_bc_total

        # Variance du small cap
        Var_R_sc_adjusted = smallcap_indice(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=y)[1]

        # Variance du big cap
        Var_R_bc_total = bigcap_indice(ppc=ppc, ph=ph, rh=rh, rb=rb, N_indice=N_indice, n_bigcap=n_bigcap, w_bigcap=w_bigcap, N_ptf=N_ptf, x=x, y=x)[1]

        # Variance du portefeuille
        Var_R = Var_R_sc_adjusted + Var_R_bc_total
        
    return E_R, Var_R

##############################################
# Delta
##############################################

def delta(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1, Precision = False):
    
    Esperance_ptf = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
    Variance_ptf = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[1]

    Esperance_indice = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y, indice = True)[0]
    Variance_indice = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y, indice = True)[1]

    Cov = covariance_RptfRindice(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y, Precision = Precision)

    Esperance = Esperance_ptf - Esperance_indice
    Var = Variance_ptf + Variance_indice - 2 * Cov
    
    return Esperance, Var

##############################################
# Poches Portefeuille
##############################################

def bigcap_ptf(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    # Espérance des Big Cap 
    E_R_bc_correct = ph * (ppc * (w_bigcap + x) + (1 - ppc) * (w_bigcap - y)) * rh  # Correct prediction outcomes
    E_R_bc_incorrect = (1 - ph) * (ppc * (w_bigcap - y) + (1 - ppc) * (w_bigcap + x)) * rb  # Incorrect prediction outcomes
    E_R_bc = (E_R_bc_correct + E_R_bc_incorrect)
    E_R_bc_total = E_R_bc * n_bigcap
    

    # Variance des Big Cap
    Var_R_bc = (ph * (rh)**2 * (ppc * (w_bigcap + x)**2 + (1 - ppc) * (w_bigcap - y)**2) + (1 - ph) * (rb)**2 * (ppc * (w_bigcap - y)**2 + (1 - ppc) * (w_bigcap + x)**2)) - E_R_bc**2
    Var_R_bc_total = Var_R_bc * n_bigcap

    return E_R_bc_total, Var_R_bc_total

def smallcap_ptf(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    # Espérance des Small Cap
    E_R_sc_ptf_correct = ((ph * ppc)/(ph * ppc + (1 - ph) * (1 - ppc))) * rh
    E_R_sc_ptf_incorrect = (1 - ((ph * ppc)/(ph * ppc + (1 - ph) * (1 - ppc)))) * rb 
    E_R_sc_ptf_total = (E_R_sc_ptf_correct + E_R_sc_ptf_incorrect) #* 1/n_smallcap_ptf * n_smallcap_ptf
    
    # Variance des Small Cap
    #Var_X_sc_ptf = ((ph * ppc)/(ph * ppc + (1 - ph) * (1 - ppc))) * ((rh - E_X_sc_ptf_total) ** 2) + (1 - ((ph * ppc)/(ph * ppc + (1 - ph) * (1 - ppc)))) * ((rb - E_X_sc_ptf_total) ** 2)
    Var_R_sc_ptf = ((((ph * ppc)/(ph * ppc + (1 - ph) * (1 - ppc))) * ((rh) ** 2) + (1 - ((ph * ppc)/(ph * ppc + (1 - ph) * (1 - ppc)))) * ((rb) ** 2)) - E_R_sc_ptf_total**2)
    Var_R_sc_ptf_N = Var_R_sc_ptf / (N_ptf - n_bigcap)

    # Espérance de l'ajustement
    E_X_ajustement = (ph * ppc + (1 - ph) * (1 - ppc)) * x - (ph * (1 - ppc) + (1 - ph) * ppc) * y
    E_W_sc = 1 - n_bigcap * (w_bigcap + E_X_ajustement)

    # Variance de l'ajustement
    Var_X_ajustement = (ph * ppc + (1 - ph) * (1 - ppc)) * (x - E_X_ajustement)**2 + (ph * (1 - ppc) + (1 - ph) * ppc) * (-y - E_X_ajustement)**2
    Var_W_sc = Var_X_ajustement * n_bigcap

    # Espérance ajustée pour les rendements "Small Cap" en prenant en compte l'ajustement de poids
    E_R_sc_ptf_adj = E_R_sc_ptf_total * E_W_sc

    # Variance ajustée pour les rendements "Small Cap" en prenant en compte l'ajustement de poids
    Var_R_sc_ptf_adj = Var_R_sc_ptf_N * E_W_sc**2 + Var_W_sc * E_R_sc_ptf_total**2 + Var_W_sc * Var_R_sc_ptf_N

    return E_R_sc_ptf_adj, Var_R_sc_ptf_adj, E_R_sc_ptf_total

def ajustement(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    # Espérance de l'ajustement
    E_x_ajustement = (ph * ppc + (1 - ph) * (1 - ppc)) * x - (ph * (1 - ppc) + (1 - ph) * ppc) * y
    E_x_total = E_x_ajustement * n_bigcap

    # Variance de l'ajustement
    Var_x_ajustement = (ph * ppc + (1 - ph) * (1 - ppc)) * (x - E_x_ajustement)**2 + (ph * (1 - ppc) + (1 - ph) * ppc) * (-y - E_x_ajustement)**2
    Var_x_ajustement_total = Var_x_ajustement * n_bigcap
    return E_x_ajustement, E_x_total, Var_x_ajustement, Var_x_ajustement_total

def esperance_W(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    
    E_Adj = ajustement(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
    return 1 - n_bigcap * (w_bigcap + E_Adj)
    
    
##############################################
# Poches Indice
##############################################

def bigcap_indice(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    # Espérance des Big Cap 
    E_R_bc_correct = ph * w_bigcap * rh  # Correct prediction outcomes
    E_R_bc_incorrect = (1 - ph) * w_bigcap * rb  # Incorrect prediction outcomes
    E_R_bc = (E_R_bc_correct + E_R_bc_incorrect)
    E_R_bc_total = E_R_bc * n_bigcap
    

    # Variance des Big Cap
    Var_R_bc = ph * w_bigcap**2 * rh**2 + (1 - ph) * w_bigcap**2 * rb**2 - E_R_bc**2
    Var_R_bc_total = Var_R_bc * n_bigcap

    return E_R_bc_total, Var_R_bc_total

def smallcap_indice(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    
    # Espérance des Small Cap de l'indice
    E_R_sc_correct_indice = ph * rh
    E_R_sc_incorrect_indice = (1 - ph) * rb
    E_R_sc_total_indice = (E_R_sc_correct_indice + E_R_sc_incorrect_indice)
    E_R_sc_total_indice_adj = E_R_sc_total_indice * (1 - n_bigcap * w_bigcap)

    # Variance des Small Cap de l'indice
    Var_R_sc_indice = (ph * (rh)**2 + (1 - ph) * (rb)**2) - E_R_sc_total_indice**2
    Var_R_sc_indice_total = Var_R_sc_indice / (N_indice - n_bigcap)
    Var_R_sc_indice_adj = Var_R_sc_indice_total * (1 - n_bigcap * w_bigcap)**2

    return E_R_sc_total_indice_adj, Var_R_sc_indice_adj

##############################################
# Espérance pour Covariances
##############################################

def calculate_permutations(combination):
    element_counts = Counter(combination).values()
    return factorial(len(combination)) // np.prod([factorial(count) for count in element_counts])


def esperance_WB(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    
    elements = {
        'rhx': [rh * w_bigcap, ph * ppc, 'x'],
        'rhy': [rh * w_bigcap, ph * (1 - ppc), 'y'],
        'rby': [rb * w_bigcap, (1 - ph) * ppc, 'y'],
        'rbx': [rb * w_bigcap, (1 - ph) * (1 - ppc), 'x'],
    }

    combinations = list(combinations_with_replacement(elements.keys(), n_bigcap))

    return sum(
        calculate_permutations(combination) * np.prod([elements[element][1] for element in combination]) *
        (sum([elements[element][0] for element in combination]) * (1 - n_bigcap * w_bigcap - 
        sum([elements[element][2] == 'x' for element in combination]) * x + 
        sum([elements[element][2] == 'y' for element in combination]) * y))
        for combination in combinations
    )

def esperance_YB(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    elements = {
        'rhx': [rh * (w_bigcap + x), rh * w_bigcap, ph * ppc, 'x'],
        'rhy': [rh * (w_bigcap - y), rh * w_bigcap, ph * (1 - ppc), 'y'],
        'rby': [rb * (w_bigcap - y), rb * w_bigcap, (1 - ph) * ppc, 'y'],
        'rbx': [rb * (w_bigcap + x), rb * w_bigcap, (1 - ph) * (1 - ppc), 'x'],
    }
    
    combinations = list(combinations_with_replacement(elements.keys(), n_bigcap))

    return sum(
        calculate_permutations(combination) * np.prod([elements[element][2] for element in combination]) *
        (sum([elements[element][0] for element in combination]) * sum([elements[element][1] for element in combination]))
        for combination in combinations
    )

def esperance_WY(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1):
    elements = {
        'rhx': [rh * (w_bigcap + x), ph * ppc, 'x'],
        'rhy': [rh * (w_bigcap - y), ph * (1 - ppc), 'y'],
        'rby': [rb * (w_bigcap - y), (1 - ph) * ppc, 'y'],
        'rbx': [rb * (w_bigcap + x), (1 - ph) * (1 - ppc), 'x'],
    }

    combinations = list(combinations_with_replacement(elements.keys(), n_bigcap))

    return sum(
        calculate_permutations(combination) * np.prod([elements[element][1] for element in combination]) *
        (sum([elements[element][0] for element in combination]) * (1 - n_bigcap * w_bigcap - 
        sum([elements[element][2] == 'x' for element in combination]) * x + 
        sum([elements[element][2] == 'y' for element in combination]) * y))
        for combination in combinations
    )

def simulate_XS(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05,y=0.1, Precision = 10000):
    n_smallcap_indice = N_indice - n_bigcap
    exposition_smallcap_indice = 1 - n_bigcap * w_bigcap
    w_smallcap_indice = exposition_smallcap_indice / n_smallcap_indice
    n_smallcap_ptf = N_ptf - n_bigcap

    results = []

    for _ in range(Precision):
        returns = np.random.choice([rh, rb], size=(n_smallcap_indice,), p=[ph, (1 - ph)])
        actions_momentum = np.where(returns == rh, 1, 0)
        probas = [ppc, (1 - ppc)]
        actions_momentum_pred = np.where(actions_momentum == 1, 
                                            np.random.choice([1, 0], size=n_smallcap_indice, p=probas), 
                                            np.random.choice([0, 1], size=n_smallcap_indice, p=probas))

        weights = np.full(n_smallcap_indice, w_smallcap_indice)
        return_smallcap_indice = np.sum(returns * weights)

        sorted_indices = np.argsort(-actions_momentum_pred)
        weights[sorted_indices[:n_smallcap_ptf]] = 1 / n_smallcap_ptf
        return_smallcap_ptf = np.sum(returns[sorted_indices[:n_smallcap_ptf]] * weights[sorted_indices[:n_smallcap_ptf]])

        XS = return_smallcap_ptf * return_smallcap_indice
        results.append(XS)

    results.append(XS)
    E_XS = np.mean(results)
    
    return E_XS

##############################################
# Covariance
##############################################

def covariance_RptfRindice(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05, y=0.1, Precision = False):
    
    if Precision == False:
        E_XW = smallcap_ptf(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_S = smallcap_indice(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_X = smallcap_ptf(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[2]
        E_WB = esperance_WB(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)
        E_Y = bigcap_ptf(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_YB = esperance_YB(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)

        E_R_ptf = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_R_indice = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y, indice = True)[0]

        return E_XW*E_S + E_X*E_WB + E_Y*E_S + E_YB - E_R_ptf*E_R_indice

    else:
        E_XW = smallcap_ptf(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_S = smallcap_indice(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_X = smallcap_ptf(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[2]
        E_WB = esperance_WB(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)
        E_Y = bigcap_ptf(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_YB = esperance_YB(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)
        E_XS = simulate_XS(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y, Precision = Precision)
        E_W = esperance_W(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)

        E_R_ptf = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y)[0]
        E_R_indice = portfolio(ppc, ph, rh, rb, N_indice, n_bigcap, w_bigcap, N_ptf, x, y, indice = True)[0]

        return E_W*E_XS + E_X*E_WB + E_Y*E_S + E_YB - E_R_ptf*E_R_indice


##############################################
# Fonction de Simulation
##############################################

def simulate_portfolio(ppc=0.6, ph=0.5, rh=1, rb=-2, N_indice=100, n_bigcap=5, w_bigcap=0.1, N_ptf=30, x=0.05,y=0.1):
    n_smallcap_indice = N_indice - n_bigcap
    exposition_smallcap_indice = 1 - n_bigcap * w_bigcap
    w_smallcap_indice = exposition_smallcap_indice/1/n_smallcap_indice
    n_smallcap_ptf = N_ptf - n_bigcap

    returns = np.random.choice([rh, rb], size=(N_indice,), p=[ph, (1 - ph)])
    actions_momentum = np.where(returns == rh, 1, 0)
    probas = [ppc, (1 - ppc)]
    actions_momentum_pred = np.where(actions_momentum == 1, 
                                     np.random.choice([1, 0], size=N_indice, p=probas), 
                                     np.random.choice([0, 1], size=N_indice, p=probas))
    actions = np.arange(N_indice)

    df_bigcap_indice = pd.DataFrame({'Actions':actions[:n_bigcap],'Returns':returns[:n_bigcap],'Momentum':actions_momentum[:n_bigcap],'Momentum_pred':actions_momentum_pred[:n_bigcap], 'Poids':w_bigcap})
    df_smallcap_indice = pd.DataFrame({'Actions':actions[n_bigcap:],'Returns':returns[n_bigcap:],'Momentum':actions_momentum[n_bigcap:],'Momentum_pred':actions_momentum_pred[n_bigcap:], 'Poids':w_smallcap_indice})
    df_indice = pd.concat([df_bigcap_indice, df_smallcap_indice])
    return_indice = np.sum(df_indice['Returns'] * df_indice['Poids'])

    df_bigcap_ptf = df_bigcap_indice.copy().assign(Poids = lambda df: df["Poids"] + df["Momentum_pred"].map({1: x, 0: -y}))
    df_smallcap_ptf = df_smallcap_indice.sort_values(by='Momentum_pred',ascending=False).head(n_smallcap_ptf).assign(Poids = (1 - df_bigcap_ptf['Poids'].sum())/1/n_smallcap_ptf)
    df_ptf = pd.concat([df_bigcap_ptf, df_smallcap_ptf])
    return_ptf = np.sum(df_ptf['Returns'] * df_ptf['Poids'])

    delta_return = return_ptf - return_indice 

    return_bigcap_ptf = np.sum(df_bigcap_ptf['Returns'] * df_bigcap_ptf['Poids'])
    return_bigcap_indice = np.sum(df_bigcap_indice['Returns'] * df_bigcap_indice['Poids'])
    return_smallcap_ptf = np.sum(df_smallcap_ptf['Returns'] * df_smallcap_ptf['Poids'])
    return_smallcap_indice = np.sum(df_smallcap_indice['Returns'] * df_smallcap_indice['Poids'])
    E_x_total = df_bigcap_ptf['Poids'].sum() - n_bigcap * w_bigcap

    W_sc = df_smallcap_ptf['Poids'].sum()
    E_WY = return_bigcap_ptf * W_sc
    
    return return_indice, return_ptf, delta_return, return_bigcap_ptf, return_smallcap_ptf,  return_bigcap_indice, return_smallcap_indice, E_x_total, W_sc, E_WY

